# A simple chi analysis

In this example we will grab some data for a simple chi analysis. First, import the relevent modules. 

In [3]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

## Part 1: Grab some data

We will grab some data from the Shillong Plateau in India. I use google maps to get the latitude and longitude of the southwest and northeast corners. Once you give the `ot_scraper` the corners and the type of data to download (in this case SRTM30 data), you can move on to downloading.

In [4]:
SH_DEM = bmt.ot_scraper(source = "SRTM30",
                        longitude_W = 91.271109, 
                        longitude_E = 92.214265, 
                        latitude_S = 25.107807, 
                        latitude_N = 25.627673,
                        prefix = "shillong")
SH_DEM.print_parameters()

The source is: SRTM30
The west longitude is: 91.271109
The east longitude is: 92.214265
The south latitude is: 25.627673
The north latitude is: 25.107807
The path is: ./
The prefix is: shillong


Now call the download. You need to wait until the `*` symbol dissappears from the right of the line (when it stops saying `In[*]`), that indicates the download has finished. 

In [5]:
SH_DEM.download_pythonic()

I am going to download the following for you:
https://portal.opentopography.org/otr/getdem?demtype=SRTMGL1&south=25.107807&north=25.627673&west=91.271109&east=92.214265&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
Finished downloading


Okay, now we need to do two things. First, we need to convert to UTM. Then we need to convert to ENVI bil format. 

In [6]:
SH_DEM.to_UTM_pythonic()
RasterFile = "shillong_SRTM30_UTM.tif"
DataDirectory = "./"
gio.convert2bil(DataDirectory, RasterFile,minimum_elevation=0)

The projections is:
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
And some extra projection information strings:
None
WGS 84
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 3395, 'height': 1872, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777778146, 0.0, 91.27097222225814,
       0.0, -0.0002777777777778146, 25.627916666662117)}
res tuple is:
(30.0, 30.0)
The destination CRS is: EPSG:32646
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 3182, 'height': 1950, 'count': 1, 'crs': CRS.from_epsg(32646), 'transform': Affine(30.0, 0.0, 325659.9621538402,
       0.0, -30.0, 2835612.1644070437)}
Removing pixels with an elevation less than 0


Okay, we now have an ENVI bil DEM. We are ready to start doing topographic analysis!

## Part 2, get some basin outlets

I want to extract some specific basins. I will get the lat-long of their outlets to select them (use *What's here* on google maps. I then create a csv containing the latitude and longitude using pandas. 

In [7]:
import pandas as pd

In [8]:
d = {'latitude': [25.187265,25.178305,25.176887,25.183027], 
     'longitude': [92.016604,91.756626,91.633770,91.378109]}
df = pd.DataFrame(data=d)
print(df)

    latitude  longitude
0  25.187265  92.016604
1  25.178305  91.756626
2  25.176887  91.633770
3  25.183027  91.378109


In [9]:
df.to_csv("basin_outlets.csv",index=False)

## Part 3: Running the analysis

In [10]:
lsdtt_parameters = {"write_hillshade" : "true", 
                    "get_basins_from_outlets" : "true", 
                    "basin_outlet_csv" : "basin_outlets.csv", 
                    "search_radius_nodes" : "15",
                    "convert_csv_to_geojson" : "true",
                    "print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}

In [11]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "shillong_SRTM30_UTM",
                                 write_prefix= "shillong_SRTM30_UTM",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: shillong_SRTM30_UTM
The write prefix is: shillong_SRTM30_UTM
The parameter dictionary is:
{'write_hillshade': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv', 'search_radius_nodes': '15', 'convert_csv_to_geojson': 'true', 'print_basin_raster': 'true', 'print_chi_data_maps': 'true'}


In [12]:
lsdtt_drive.run_lsdtt_command_line_tool()

Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.


## Part 3: Plotting the data

First, lets just see where we are.

In [13]:
DataDirectory = "./"
Base_file = "shillong_SRTM30_UTM"  
img_name = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth")

Your colourbar will be located: right
xsize: 3182 and y size: 1950
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset is projected.
The projection string is: WGS 84 / UTM zone 46N
WGS 84 / UTM zone 46N
This seems to be from the new gdal version
Zone string is: 46N
The zone is: 46
And the hemisphere is: N
The EPSG string is: epsg:32646
epsg:32646
minimum values are x: 325659.0 and y: 2777112.0
I made the ticks.
x labels are: 
['330', '340', '350', '360', '370', '380', '390', '400', '410']
x locations are:
[330000.0, 340000.0, 350000.0, 360000.0, 370000.0, 380000.0, 390000.0, 400000.0, 410000.0]
y labels are: 
['2780', '2790', '2800', '2810', '2820', '2830']
y locations are:
[2780000.0, 2790000.0, 2800000.0, 2810000.0, 2820000.0, 2830000.0]
This colourmap is: gray
The number of axes are: 1
Axes(0,0;1x1)
Axes(0,0;1x1)
N axes are: 1
Axes(0,0;1x1)
xsize: 3182 and y size: 1950
NoData is: -9999.0
Let me get that projection for you
In

findfont: Font family ['Liberation Sans'] not found. Falling back to DejaVu Sans.


In [14]:
img_name

'./shillong_SRTM30_UTM_hillshade.png'

<img src="shillong_SRTM30_UTM_hillshade.png" width=600>

Now plot the channels

In [18]:
ChannelFileName = "shillong_SRTM30_UTM_chi_data_map.csv"
channel_img_name = lsdmw.PrintChannels(DataDirectory,Base_file,ChannelFileName, 
                               add_basin_labels = True, cmap = "gist_earth", cbar_loc = "right", 
                               size_format = "geomorphology", dpi = 300, 
                               plotting_column = "basin_key")

Let me get the point data
The filename is: ./shillong_SRTM30_UTM_chi_data_map.csv
The object file prefix is: shillong_SRTM30_UTM_chi_data_map
Warning, you are using an experimental version of LSDMT that is implementing Pandas dataframe to improve the performance. It is still unstable, switch PANDEX to False in your PointData parameters to use the regular way
Loading your file from csv
done
The parameter names are
['latitude', 'longitude', 'chi', 'elevation', 'flow_distance', 'drainage_area', 'source_key', 'basin_key']
Your colourbar will be located: None
xsize: 3182 and y size: 1950
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset is projected.
The projection string is: WGS 84 / UTM zone 46N
WGS 84 / UTM zone 46N
This seems to be from the new gdal version
Zone string is: 46N
The zone is: 46
And the hemisphere is: N
The EPSG string is: epsg:32646
epsg:32646
minimum values are x: 325659.0 and y: 2777112.0
I made the ticks.
x l

/usr/local/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

I got the easting and northing
I got the data column you wanted
I also got the data for scaling, which is in column drainage_area
The size of the array is: 
(46056,)
I am going to convert data to log for point scaling.
I logged (is it a verb?) your scaled data, the minimum is 5.954242509439325 and the maximum is 8.918873978335956 but all the values inferior to -10 will be -10
I am scaling your points for you
max is: 8.918873978335956 and min is: 5.954242509439325
I have got a scaled point array,
I will plot the points now.
The colourmap is: gist_earth
I will colour by the points
The aspect ratio is: 1.6317948717948718
I need to adjust the spacing of the colourbar.
The cbar characters are: 0 and the cbar text width is: 0.4
No colourbar
The figure size is: 
[6.25, 4.036675047140163]
Map axes are:
[0.136, 0.16102361285199346, 0.8319999999999999, 0.789430660116624]
cbar_axes are:
None
Number of axes are: 1


In [16]:
channel_img_name

'./shillong_SRTM30_UTM_channels_coloured_by_basin.png'

<img src="shillong_SRTM30_UTM_channels_coloured_by_basin.png" width=600>

Now show the basins and the channels

In [19]:
basin_img_name = lsdmw.PrintChannelsAndBasins(DataDirectory,Base_file,
                                              add_basin_labels = True, 
                                              cmap = "jet", cbar_loc = "right", 
                                              size_format = "geomorphology", dpi = 300)

Basin keys are: 
[0, 1, 2, 3]
I'm going to make the basin plots. Your topographic data must be in ENVI bil format or I'll break!!
shillong_SRTM30_UTM_AllBasins.bil


NameError: name 'LSDV' is not defined

In [20]:
basin_img_name = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,
                                           use_keys_not_junctions = True, 
                                           show_colourbar = False,cmap = "jet", 
                                           colorbarlabel = "colourbar", size_format = "geomorphology",
                                           fig_format = "png", dpi = 250, 
                                           include_channels = True, label_basins = True):

SyntaxError: invalid syntax (<ipython-input-20-cf911e2ff448>, line 6)